In [1]:
! pip install transformers

from transformers import AutoTokenizer, AutoModelForMaskedLM
from google.colab import drive
from pandas import DataFrame
import pandas as pd
import os
import torch
import time

drive.mount('/content/drive')

folder = "drive/Othercomputers/Il mio Laptop/Universita/[IA] Artificial Intelligence/[HLT] Human Language Technologies/project/"
bertModel  = "dbmdz/bert-base-italian-xxl-cased"

# start_time = time.time()
# print("--- %s seconds ---" % (time.time() - start_time))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 27.9 MB/s 
     |████████████████████████████████| 7.6 MB 57.2 MB/s 
     |████████████████████████████████| 163 kB 96.1 MB/s 
Mounted at /content/drive


In [2]:
dataset = pd.read_csv(folder+"Sources/dataset.csv")

In [3]:
dataset

,text,labels
0,"Non ipertensione , dislipidemia o diabete .",O B-DISO O B-DISO O B-DISO O
1,Nel 2003 operata di sostituzione valvolare per...,O O O O O O O B-DISO I-DISO O O O O O O O O O O
2,Agli inizi del 2006 frattura traumatica di rad...,O O O O B-DISO O O O O O O O O O O O O O O O ...
3,Nel settembre 2006 frattura traumatica di femo...,O O O B-DISO O O O O O O O O O O O O O O O O O O
4,riconoscimento di osteoporosi con mancato cons...,O O B-DISO O O O O B-DISO O
...,...,...
69925,Si segnala all' ecg recidiva dell' aritmia da ...,O O O B-TREA O O O O O O O O O
69926,In considerazione delle condizioni cliniche ge...,O O O O O O O O O O O O O O O O O O B-TREA O ...
69927,Tra circa 3 mesi ripetere un ecg dinamico di 2...,O O O O O O B-TREA O O O O O O O
69928,Setto nasale lievemente deviato verso sn. Rino...,B-BODY I-BODY O O O O O O


In [4]:
labels_to_ids = {'B-ACTI': 0, 'B-BODY': 1, 'B-DISO': 2, 'B-DRUG': 3, 'B-SIGN': 4, 'B-TREA': 5, 'I-ACTI': 6, 'I-BODY': 7, 'I-DISO': 8, 'I-DRUG': 9, 'I-SIGN': 10, 'I-TREA': 11, 'O': 12}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(bertModel)

In [7]:
def align_label(token: list, labels: str):
  # Trasforming a string of label into array
  labels = labels.split()

  # We take into cosideration the previous word to identify if the id is already seen
  previous_id = None
  # Aligned labels
  label_ids = []

  # We can all ids in the token and we try to associate to a label  
  for word_idx in token:

    # typically when we encounter [CLS]
    if word_idx is None: 
      label_ids.append(-100)
    else:
      # We try to associate a label
      try:
        label_ids.append(labels_to_ids[labels[word_idx]])
      except:
        label_ids.append(-100)
    
    previous_id = word_idx 
    
  return label_ids

In [8]:
texts,labels = [],[]
for _,row in dataset.iterrows():
  token_text = tokenizer(row[0], padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
  label_ids = align_label(token_text.word_ids(),row[1])

  texts.append(token_text)
  labels.append(label_ids)

In [7]:
# tokenized_texts = []
# for text in dataset['text'].values.tolist():
#   tokenized_texts.append(tokenizer(text,padding='max_length', max_length = 512, truncation=True, return_tensors="pt"))  

In [8]:
# text = dataset.iloc[4].text
# label = dataset.iloc[4].labels
# tokens = tokenizer(text,padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
# r = align_label(text,label)

In [9]:
# texts = []
# for text in dataset['text'].values.tolist():
#   texts.append(tokenizer(text,padding='max_length', max_length = 512, truncation=True, return_tensors="pt"))  

In [10]:
# labels = []
# for i,j in zip(text, dataset['labels'].values.tolist()):
#   labels.append(align_label(i,j))

In [11]:
# class DataSequence(torch.utils.data.Dataset):

#     def __init__(self, df):

#         lb = [i.split() for i in df['labels'].values.tolist()]
#         txt = df['text'].values.tolist()
#         self.texts = [tokenizer(str(i),
#                                padding='max_length', max_length = 512, truncation=True, return_tensors="pt") for i in txt]
#         self.labels = [align_label(i,j) for i,j in zip(txt, lb)]

#     def __len__(self):

#         return len(self.labels)

#     def get_batch_data(self, idx):

#         return self.texts[idx]

#     def get_batch_labels(self, idx):

#         return torch.LongTensor(self.labels[idx])

#     def __getitem__(self, idx):

#         batch_data = self.get_batch_data(idx)
#         batch_labels = self.get_batch_labels(idx)

#         return batch_data, batch_labels